# Functions

In [6]:
def f(x):
    return x**2

In [7]:
f(5)

25

In [21]:
def pow(x,y=2):
    return x**y
    print('I will never be executed')

In [22]:
pow(3, 3), pow(3), pow(y=1, x=42)

(27, 9, 42)

Parameters can be optional if a default value is provided. Optional parameters must be at the end of the function definition.

The function exists at the return statement, code afterwards is not executed.

In [13]:
def void():
    print('hello')

In [16]:
out = void()
print(out)

hello
None


Functions without return statement return None.

In [18]:
def dummy_function():
    pass

If a function (or class) has no content, the pass statement is required,

# Classes

In [120]:
class Vehicle:
    """example class"""
    def __init__(self, wheels, seats):
        """
        The __init__ method is used for object creation.
        """
        self.wheels = wheels
        self.seats = seats
    def drive(self, km):
        print("driving {} km on {} wheels".format(km, self.wheels))

In [121]:
bicycle = Vehicle(2, 1)
bicycle.drive(10)

driving 10 km on 2 wheels


In [122]:
bicycle.wheels

2

## Inheritance

In [128]:
class Car(Vehicle):
    def __init__(self, seats, power):
        super().__init__(wheels=4, seats=seats) # calls constructuor of parent class
        self.power = power
    def drive(self, km):
        print("driving {} km on {} wheels with {} kW power".format(
            km, self.wheels, self.power))

In [129]:
my_car = Car(5, 150)
my_car.drive(200)

driving 200 km on 4 wheels with 150 kW power


If a method is overwritten in the child class (like init in the example), the method of the parent class can be accessed with super().

### Multiple Inheritance

In [169]:
class PurchaseMixin:
    price = None
    currency = 'EUR'
    nr_owned = 0
    def buy(self, nr=1):
        self.nr_owned += nr
        print("purchased {} piece(s) for {} {}".format(nr, nr*self.price, self.currency))
    def sell(self, nr=1):
        if nr > self.nr_owned:
            raise ValueError('you own less than you want to sell!')
        self.nr_owned -= nr
        print("sold {} piece(s) for {} {}".format(nr, nr*self.price, self.currency))

In [170]:
class PurchasableCar(PurchaseMixin, Car):
    pass

In [171]:
my_cars = PurchasableCar(5, 150)
my_cars.price = 30000
my_cars.buy(2)
my_cars.sell()
my_cars.drive(250)
my_cars.nr_owned

purchased 2 piece(s) for 60000 EUR
sold 1 piece(s) for 30000 EUR
driving 250 km on 4 wheels with 150 kW power


1

Classes in Python may inherit from multiple parent classes. This can be used to add additional features to classes.

### Naming Resolution

In [218]:
class A:
    def __init__(self):
        self.i = 1
        print('init A')
    def func(self, x):
        return x+1
class B:
    def __init__(self):
        self.i = 2
        print('init B')
    def func(self, x):
        return x+2

In [219]:
class AB(A,B):
    pass
class BA(B,A):
    pass

In [220]:
ab = AB()
ba = BA()
ab.func(0), ba.func(0)

init A
init B


(1, 2)

If parent classes have the same methods (including init), the derived class uses the method of the 1st parent.

In [221]:
class AB2(A, B):
    def func(self,x):
        return B.func(self, x)

In [222]:
ab2 = AB2()
ab2.func(0)

init A


2

In

## Property Methods

In [172]:
my_cars.price = 'hello'
my_cars.buy() # output is not what is expected

purchased 1 piece(s) for hello EUR


In languages like C++ and Java one usually implements getter and setter methods for any public property. 
This is discouraged in Python because it creates a lot of boilerplate code.
Instead, properties should be accessed directly.

If it turns out later that logic (like validations, formatting) is required when getting or setting properties, they can be changed in the class definition to property methods without changing the class api.

In [176]:
from numbers import Number
class SafePurchaseMixin(PurchaseMixin):
    _price = None
    
    @property
    def price(self):
        if self._price is None:
            raise ValueError('price not set')
        return self._price
    
    @price.setter
    def price(self, x):
        if not isinstance(x, Number):
            raise TypeError('Price must be numeric')
        if x <= 0:
            raise ValueError('Price must be positive')
        self._price = x
        
    @price.deleter # for completeness, often not required
    def price(self):
        self._price = None

In [177]:
class PurchasableCar2(SafePurchaseMixin, Car):
    pass

In [178]:
my_cars = PurchasableCar2(5, 150)
my_cars.price = 30000 # note that the api is identical to PurchasableCars
my_cars.buy(2)
my_cars.sell()
my_cars.drive(250)
my_cars.nr_owned

purchased 2 piece(s) for 60000 EUR
sold 1 piece(s) for 30000 EUR
driving 250 km on 4 wheels with 150 kW power


1

In [179]:
try:
    my_cars.price = 'hello'
except TypeError as e:
    print(e)

Price must be numeric


In [180]:
del my_cars.price
try:
    print(my_cars.price)
except ValueError as e:
    print(e)

price not set


In the property methods for price, validation has been included without any change of the class api.

## Dunder Methods

In [116]:
from math import sqrt
class ComplexNumber:
    """
    just as an example, in practice use the built-in complex data type!
    """
    def __init__(self,real, imag=0):
        self.r = real
        self.i = imag
    
    @staticmethod
    def _convert_num(x):
        """
        Converts number or ComplexNumber into real and imaginary part.
        This is an internal method, marked by leading underscore (_).
        Furhermore, it is a static method, i.e. it does not require any input from
        the object itself (as indicated with missing self parameter). 
        Static methods require the @staticmethod decorator.
        """
        if isinstance(x, ComplexNumber):
            r2 = x.r
            i2 = x.i
        else:
            r2 = x
            i2 = 0
        return r2, i2
    
    def __str__(self):
        """Defines string representation of the object."""
        return "{r} + {i} * i".format(r=self.r, i=self.i)
    
    def __repr__(self):
        """Defines a printable representation of the object. 
        Usually __repr__ contains more debugging information than __str__."""
        return "ComplexNumber object, value: {r} + {i} * i".format(r=self.r, i=self.i)
    
    def __add__(self, x):
        r2, i2 = self._convert_num(x)
        return ComplexNumber(self.r + r2, self.i + i2)

    def get_abs_value(self):
        return sqrt(self.r**2 + self.i**2)

In [114]:
c1 = ComplexNumber(1, 2)
print(c1)

1 + 2 * i


The print function implicitly converts the ComplexNumber object into a string, i.e. printing the output of the 
    
    __str__ 
method.

In [115]:
c1

ComplexNumber object, value: 1 + 2 * i

In [106]:
c1.r, c1.i, c1.get_abs_value()

(1, 2, 2.23606797749979)

In [107]:
c2 = ComplexNumber(2, 3)
c3 = c1 + c2
print(c3)

3 + 5 * i


In [108]:
print(c1 + 7)

8 + 2 * i


# Namespaces

In [43]:
outside1 = 42
outside2 = 0
def f_bad(x):
    outside2 = x # this is actually a local variable 
                 # which has nothing to do with outside 2 defined before
    print(outside2)
    return x + outside1 # extremely dangerous usage of global variable

In [37]:
outside2

0

In [44]:
f_bad(7), outside2

7


(49, 0)

Variables defined inside a function (or class) are local, i.e. they are only valid inside the function and changes of them do not influence anything outside the function. This is true even if outside variables have the same name (here global2).

Variables defined outside the function, however, can be accessed from inside the function. This is however very dangerous and should be avoided!

In [47]:
def f_even_worse(x):
    global outside2 # makes the variable defined outside changable inside the function
    outside2 = x # the outside variable is actually changed here!
    print(outside2)
    return x + outside1 # extremely dangerous usage of global variable

In [48]:
outside2

0

In [50]:
f_even_worse(7), outside2

7


(49, 7)

The *global* stament makes the variable defined outside changable inside the function. The usage of this pattern is __extremely__ dangerous and must be avoided!

In [45]:
def f_good(x, i):
    print(x)
    return x + i

In [46]:
f_good(7, 42)

7


49

Refactored version of the previous function without implicit usage of variable defined outside the function.

In [60]:
def f_mod_input(i: int, l: list):
    l.append(i)
    i = i*2
    return i

In [63]:
i1 = 42 # basic variable
l1 = [1, 2] # list object
f_mod_input(i1, l1)

84

In [64]:
i1, l1

(42, [1, 2, 42])

Both function parameters (here integer and list) are changed inside the function.

* The basic type (here integer) is however not changed in the outside scope - call by value
* The complex type (here list) is changed in the outside scope by the modification inside the function - call by reference

It is a common source of errors and (in my opinion) a major weakness of Python language that call by value and call by reference are not clearly separated by syntax (like in C (++) where pointers have * as prefix).

Call by reference can be used on purpose to modify objects in outer scope by the function. However, it is safer to explicitly create a new object inside the function, modify that and return it.

In [73]:
def f_safer(i: int, l: list):
    l2 = l.copy() # creates new list object
    l2.append(i)
    i = i*2
    return i, l2

In [74]:
i1 = 42 # basic variable
l1 = [1, 2] # list object
f_safer(i1, l1)

(84, [1, 2, 42])

In [75]:
i1, l1

(42, [1, 2])

# Error Handling

Author: Benjamin Lungwitz